# PROBLEM STATEMENT 

The problem statement addressed by the code is to develop a computer-aided system for detecting tumors.
Here's a more specific breakdown:
# Given
A dataset of medical images containing both tumor and non-tumor regions.
# Objective 
Develop a CNN model that can accurately detect unseen medical images as containing a tumor or not.
# Desired Outcome 
The trained model should be able to analyze new medical images and provide a reliable prediction of the presence or absence of a tumor, potentially aiding medical professionals in early detection and diagnosis.

# Let's Import Libraries

In [ ]:
import os
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('dark_background')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# One Hot Encoding Target Class

In [ ]:
# Tumor-0
# Non- Tumor-1

encoder= OneHotEncoder()
encoder.fit([[0], [1]])



# Creating  3 Lists --
1. data list for storing image data in numpy array form
2. paths list for storing paths of all images
3. result list for storing one hot encoded form of target class whether normal or tumor

In [ ]:
# Cell updates result list for tumor images

data= []
paths= []
result= []

for r, d, f in os.walk(r'../input/brain-mri-images-for-brain-tumor-detection/yes'):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r, file))
            

for path in paths:
    img= Image.open(path)
    img= img.resize((128, 128))
    img= np.array(img)
    if (img.shape==(128, 128, 3)):
        data.append(np.array(img))
        result.append(encoder.transform([[0]]).toarray())

In [ ]:
# cel result list with non-tumor images

paths= []
for r, d, f in os.walk(r"../input/brain-mri-images-for-brain-tumor-detection/no"):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r, file))
    
    for path in paths:
        img= Image.open(path)
        img= img.resize((128, 128))
        img= np.array(img)
        if (img.shape==(128, 128, 3)):
            data.append(np.array(img))
            result.append(encoder.transform([[1]]).toarray())

In [ ]:
data= np.array(data)
data.shape

In [ ]:
result= np.array(result)
result= result.reshape(139, 2)
result.shape

# Splitting data into Training & Testing

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(data, result, test_size= 0.2, shuffle= True, random_state=0)

# Model Building 

Batch normalization is a technique for training very deep neural networks that standardizes the inputs to a layer for each mini-batch. This has the effect of stabilizing the learning process and dramatically reducing the number of training epochs required to train deep networks

In [ ]:
model= Sequential()

model.add(Conv2D(32, kernel_size=(2, 2), input_shape=(128, 128, 3), padding='Same'))
model.add(Conv2D(32, kernel_size= (2, 2), activation='relu', padding='Same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size= (2, 2), activation= 'relu', padding='Same'))
model.add(Conv2D(64, kernel_size= (2, 2), activation='relu', padding='Same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size= (2, 2), strides= (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss= 'categorical_crossentropy', optimizer='Adamax')
print(model.summary())

In [ ]:
y_train.shape

In [ ]:
history= model.fit(x_train, y_train, epochs= 30, batch_size= 40, verbose= 1, validation_data= (x_test, y_test))

# Plotting Losses

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Test', 'Validation'], loc='upper right')
plt.show()

# Let's check the Model

In [ ]:
def names(number):
    if number==0:
        return 'Its a Tumor'
    else:
        return 'Non Tumor'

In [ ]:
from matplotlib.pyplot import imshow
img = Image.open(r"../input/brain-mri-images-for-brain-tumor-detection/no/N17.jpg")
x = np.array(img.resize((128,128)))
x = x.reshape(1,128,128,3)
res = model.predict_on_batch(x)
classification = np.where(res == np.amax(res))[1][0]
imshow(img)
print(str(res[0][classification]*100) + '% Confidence This Is ' + names(classification))

In [ ]:
from matplotlib.pyplot import imshow
img = Image.open(r"../input/brain-mri-images-for-brain-tumor-detection/yes/Y3.jpg")
x = np.array(img.resize((128,128)))
x = x.reshape(1,128,128,3)
res = model.predict_on_batch(x)
classification = np.where(res == np.amax(res))[1][0]
imshow(img)
print(str(res[0][classification]*100) + '% Confidence This Is A ' + names(classification))

# Done, Thank you. If you find it impressive Please UpVote..!